I plan to update the model in a timely manner. 

original code

https://www.kaggle.com/code/swimmy/lgbm-opt-model-jpx

# **if it is useful for you, please vote!**

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
prices["Date"]

In [ ]:
NDAYS = 34
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","Avg","High","Open","Close","Low","Volume"]]
y=prices[["Target"]]
codes = X.SecuritiesCode.unique()

In [ ]:
import optuna

def objectives(trial):
    params = {
#             'num_leaves': trial.suggest_int('num_leaves', 200, 3000),
#             'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
#             'max_bin': trial.suggest_int('max_bin', 2, 100),
#             'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'subsample': trial.suggest_uniform('subsample',0.6, 1),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree',0.6, 1)
        
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

last best param with range

'num_leaves': trial.suggest_int('num_leaves', 200, 3000),


            'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
            
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),

LGBMRegressor(learning_rate=0.6925315900777659, max_bin=89, n_estimators=1786,
              num_leaves=2667, random_seed=16)
              
              lb 5.434

In [ ]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objectives, n_trials=35)

# 最適パラメータ取得
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 16
params_best['learning_rate'] = 0.6925315900777659
params_best['max_bin'] = 89
params_best['n_estimators'] = 1786
params_best['num_leaves'] = 2667
    
# 最適パラメータで学習/予測    
model_o = LGBMRegressor(**params_best)#

In [ ]:
model_o.fit(X,y)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    df = sample_prediction[["Date","SecuritiesCode","Avg"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df["Volume"] = prices["Volume"]
    print(df["Date"])
#     ,"High","Open","Close","Low","Volume"
    df.Date = pd.to_datetime(df.Date)
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)